In [2]:
!pip install ipywidgets==7.0.0 --quiet

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.0.1 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.0.1 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.
sparkmagic 0.20.4 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.7 which is incompatible.
spyder 4.0.1 requires jedi==0.14.1, but you have jedi 0.18.2 which is

In [3]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

In [4]:
model_id, model_version = "xgboost-classification-model", "*"

In [5]:
from sagemaker import image_uris, model_uris, script_uris


from sagemaker import image_uris, instance_types, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
inference_image_uri = image_uris.retrieve(framework='xgboost',region='us-west-2',version='1.5-1', 
                                          image_scope="inference")

deploy_source_uri = script_uris.retrieve(
    model_id="xgboost-classification-model", model_version="*", script_scope="inference"
)
print(deploy_source_uri)
# Retrieve the model uri.
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)




/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


s3://jumpstart-cache-prod-us-west-2/source-directory-tarballs/xgboost/inference/classification/v1.1.0/sourcedir.tar.gz


In [19]:
 sagemaker_client = boto3.client(
        service_name="sagemaker", region_name=boto3.Session().region_name
    )
session = sagemaker.session.Session(
        sagemaker_client=sagemaker_client
    )


In [20]:
!pip download   -d "AI21brokerInferenceCode/lib/botocore/" --no-deps botocore==1.29.162 

!pip download   -d "AI21brokerInferenceCode/lib/boto3/" --no-deps boto3==1.26.162 

  File was already downloaded /root/DemoNotebooks/AI21_summary_Inference/AI21brokerInferenceCode/lib/botocore/botocore-1.29.162-py3-none-any.whl
Successfully downloaded botocore
  Obtaining dependency information for boto3==1.26.162 from https://files.pythonhosted.org/packages/38/cb/0de1d433d5d2dfb0adf022a74778c1f630d5c4ae846c093d65083b601ed2/boto3-1.26.162-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 1.7 MB/s eta 0:00:00:00:01
Saved ./AI21brokerInferenceCode/lib/boto3/boto3-1.26.162-py3-none-any.whl
Successfully downloaded boto3


In [12]:
endpoint_name = "ai21-broker-ep"
model = Model(
        image_uri=inference_image_uri,
        source_dir="/root/DemoNotebooks/AI21_summary_Inference/AI21brokerInferenceCode",
        model_data=model_uri,
        entry_point="inference.py",  # entry point file in source_dir and present in deploy_source_uri
        role=aws_role,
        predictor_cls=Predictor,
        name=endpoint_name,
        sagemaker_session=session,
    )

In [13]:
pwd

'/root/DemoNotebooks/AI21_summary_Inference'

In [14]:
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig



# uncomment the below code for real time inference deployment

# model_predictor = model.deploy(
#     initial_instance_count=1,
#     instance_type="ml.m5.large",
#     predictor_cls=Predictor,
#     endpoint_name=endpoint_name,
#     serverless_inference_config= serverless_inf_config
# )
# model_predictor = model.deploy(
#     predictor_cls=Predictor,
#     endpoint_name=endpoint_name
# )

In [15]:
# the below code for serverless inference deployment
serverless_inf_config = ServerlessInferenceConfig(memory_size_in_mb=2048, max_concurrency=5, provisioned_concurrency=None)

model_predictor = model.deploy(
    predictor_cls=Predictor,
    endpoint_name=endpoint_name,
    serverless_inference_config= serverless_inf_config
)

----!

In [16]:
import boto3,json
def query_endpoint(encoded_text, endpoint_name):
    client = boto3.client("runtime.sagemaker",region_name='us-west-2')
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="text/csv", Body=encoded_text
    )
    return response


def parse_response(query_response):
    #model_predictions = json.loads(query_response["Body"].read())
    #generated_text = model_predictions["generated_text"]
    return query_response["Body"].read()

In [17]:
query = "I like this so much more than the Voyage. The shape makes for easier holding. I only wish this devise was maybe 2 inches taller then it would look like a real book|Replacing older reader without a light and traveling overseas soon. With electronics in the cabin under scrutiny  this very small and very lightweight reader fits the bill! Perfect in small purses and large pockets. I can read without annoying seatmate with cabin overhead seat light. Easy to hold in either hand and screen adjusts for whichever direction you're holding it.|This kindle is awesome. I love the design of it. Nothing software wise that was different than my other kindle but the way it's designed makes it much more comfortable and easier to read. The only reason I didn't give it five stars was because of the price. If it were cheaper  this kindle would have been five stars no doubt.|The new Kindle is a great gift and fantastic entertainment idea.|Got me reading again! Love the merlot cover. This one is the smallest of all the kindle products I believe  and that's a big reason I picked. Also the lightest I believe. Battery life is forever.Two negative points are the Amazon logo in the cover  and the fact you can't do inverted text (black background white text).Those 2 points dropped it from 4 to 5 star for me.Still highly recommended.|This is not an upgrade by any means! My three year old kindle outperformed Oasis.Battery life better than a week with the lights on lowest setting Magnetic connector is poorly designed and grows weak Two pieces to keep up with; unsnaps constantly Amazon Customer Services promises it will resolved in an update|Its a great ereader but a little pricy. It has the same specs as the Voyage as far as pixels and procesor  which costs less. The only difference between this and the Voyage  is that it's much lighter  it's brighter (LEDs)  and a longer battery life if you're using the cover. I've noticed the battery life drains much faster without the cover  but you'll still be able to read for hours without it. You would think it be water resistant  hence the name but it isn't. But then I don't plan on reading when it's raining or outside or while taking a bath. Overall  it's the best ereader Amazon currently has to offer but if you don't mind the price tag and want the best  then go for it! Voyage is still not a bad option though|Small  light  but comfortable to hold with either hand. If you keep it in airplane mode with medium lighting  the battery last for weeks. Lovin' it so far.|Almost feels like reading a book in a light comforting way. Easy to carry around. Great on eyes and reading is enjoyable|Easy to handle. Light weight. I like that it automatically shuts off when the cover is closed."

In [18]:
endpoint_name = "ai21-broker-ep"
query_response = query_endpoint(query, endpoint_name=endpoint_name)

generated_text = parse_response(query_response)
print(
        generated_text
    )

b"This kindle is awesome  I love the design of it  nothing software wise that was different from my other kindle but the way it's designed makes it much more comfortable and easier to read. The only reason I didn't give it five stars was because of the price..It's a great ereader but a little pricy. It has the same specs as the Voyage as far as pixels and procesor which costs less but it's much lighter  brighter (LEDs) and a longer battery life if you're using the cover."


In [19]:
import shutil 
import os.path

# Creating the ZIP file 
archived = shutil.make_archive('/root/DemoNotebooks/AI21_summary_Inference', 'zip', '/root/DemoNotebooks/AI21_summary_Inference')

if os.path.exists('AI21_summary_Inference.zip'):
   print(archived) 
else: 
   print("ZIP file not created")

/root/DemoNotebooks/AI21_summary_Inference.zip
